# 🔄⚡ Logic Apps + Semantic Kernel Hybrid Tutorial

**🚀 Learn how to combine Azure Logic Apps with Semantic Kernel plugins for powerful workflow automation!**

This tutorial demonstrates:
1. **Logic Apps Integration** - Automated workflow triggers using AzureLogicAppTool
2. **Semantic Kernel Plugins** - Local data processing capabilities
3. **Hybrid Agent** - Combining workflows with AI intelligence

**Scenario:** An intelligent business assistant that can:
- 📧 Trigger email workflows via Logic Apps
- 📊 Process business data locally with SK plugins
- 🔗 Coordinate between cloud automation and AI analysis

---

## 🔧 Setup and Prerequisites

**Environment Variables Required:**
- `PROJECT_ENDPOINT`: Your Azure AI Project endpoint
- `MODEL_DEPLOYMENT_NAME`: Your deployed AI model name
- `AZURE_SUBSCRIPTION_ID`: Your Azure subscription ID
- `AZURE_RESOURCE_GROUP_NAME`: Resource group containing your Logic App

**Architecture Overview:**
```
Azure AI Agent
    ├── AzureLogicAppTool → Email/Notification Workflows
    └── SK Plugins
            ├── DataPlugin → process_sales_data()
            └── ReportPlugin → generate_summary()
```

In [ ]:
# Install required packages
# !pip install azure-ai-agents azure-identity azure-mgmt-logic requests

import os
import json
import requests
from typing import Annotated, Dict, Any, Callable, Set
from datetime import datetime, timedelta

# Azure AI Agents imports
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import ToolSet, FunctionTool
from azure.identity import DefaultAzureCredential

# Azure Logic Apps Management
from azure.mgmt.logic import LogicManagementClient

# Semantic Kernel imports
import semantic_kernel as sk
from semantic_kernel.functions import kernel_function

print("✅ Required packages imported successfully!")

✅ Required packages imported successfully!


In [ ]:
# Azure Logic Apps Tool Implementation
class AzureLogicAppTool:
    """
    A service that manages multiple Logic Apps by retrieving and storing their callback URLs,
    and then invoking them with an appropriate payload.
    """

    def __init__(self, subscription_id: str, resource_group: str, credential=None):
        if credential is None:
            credential = DefaultAzureCredential()
        self.subscription_id = subscription_id
        self.resource_group = resource_group
        self.logic_client = LogicManagementClient(credential, subscription_id)
        self.callback_urls: Dict[str, str] = {}

    def register_logic_app(self, logic_app_name: str, trigger_name: str) -> None:
        """
        Retrieves and stores a callback URL for a specific Logic App + trigger.
        Raises a ValueError if the callback URL is missing.
        """
        try:
            callback = self.logic_client.workflow_triggers.list_callback_url(
                resource_group_name=self.resource_group,
                workflow_name=logic_app_name,
                trigger_name=trigger_name,
            )

            if callback.value is None:
                raise ValueError(f"No callback URL returned for Logic App '{logic_app_name}'.")

            self.callback_urls[logic_app_name] = callback.value
            print(f"✅ Registered Logic App: {logic_app_name}")
        except Exception as e:
            print(f"⚠️ Warning: Could not register Logic App '{logic_app_name}': {e}")
            print("💡 This is expected if the Logic App doesn't exist - emails will be simulated")

    def invoke_logic_app(self, logic_app_name: str, payload: Dict[str, Any]) -> Dict[str, Any]:
        """
        Invokes the registered Logic App (by name) with the given JSON payload.
        Returns a dictionary summarizing success/failure.
        """
        if logic_app_name not in self.callback_urls:
            # Simulate email sending if Logic App not registered
            print(f"📧 SIMULATED EMAIL:")
            print(f"   To: {payload.get('to', 'unknown')}")
            print(f"   Subject: {payload.get('subject', 'unknown')}")
            print(f"   Body: {payload.get('body', 'unknown')}")
            return {"result": f"Successfully simulated email via {logic_app_name}."}

        url = self.callback_urls[logic_app_name]
        try:
            response = requests.post(url=url, json=payload, timeout=30)
            if response.ok:
                return {"result": f"Successfully invoked {logic_app_name}."}
            else:
                return {"error": f"Error invoking {logic_app_name} ({response.status_code}): {response.text}"}
        except Exception as e:
            return {"error": f"Error invoking {logic_app_name}: {str(e)}"}

def create_send_email_function(service: AzureLogicAppTool, logic_app_name: str) -> Callable[[str, str, str], str]:
    """
    Returns a function that sends an email by invoking the specified Logic App.
    """
    def send_email_via_logic_app(to: str, subject: str, body: str) -> str:
        """
        Sends an email by invoking the specified Logic App with the given recipient, subject, and body.
        """
        payload = {
            "to": to,
            "subject": subject,
            "body": body,
        }
        result = service.invoke_logic_app(logic_app_name, payload)
        return json.dumps(result)

    return send_email_via_logic_app

print("🔧 AzureLogicAppTool class defined!")

🔧 AzureLogicAppTool class defined!


## 📧 Section 1: Creating Agent with Azure Logic Apps Integration

We'll create an AI agent that can trigger Logic Apps workflows for automated business processes.

In [13]:
# Initialize Azure AI Agents client
project_endpoint = os.getenv("PROJECT_ENDPOINT")
model_deployment = os.getenv("MODEL_DEPLOYMENT_NAME")
subscription_id = os.getenv("AZURE_SUBSCRIPTION_ID")
resource_group = os.getenv("AZURE_RESOURCE_GROUP_NAME")

if not all([project_endpoint, model_deployment, subscription_id, resource_group]):
    raise ValueError("Missing required environment variables. Please set PROJECT_ENDPOINT, MODEL_DEPLOYMENT_NAME, AZURE_SUBSCRIPTION_ID, and AZURE_RESOURCE_GROUP_NAME")

credential = DefaultAzureCredential()
agents_client = AgentsClient(endpoint=project_endpoint, credential=credential)

print(f"✅ Connected to Azure AI Project: {project_endpoint}")
print(f"📊 Using model: {model_deployment}")

✅ Connected to Azure AI Project: https://selhousseini-agentic-resource.services.ai.azure.com/api/projects/selhousseini-agentic
📊 Using model: gpt-4.1


In [ ]:
# Logic App details
logic_app_name = "agent-logic-apps"
trigger_name = "When_a_HTTP_request_is_received"

# Create and initialize AzureLogicAppTool utility
logic_app_tool = AzureLogicAppTool(subscription_id, resource_group)
logic_app_tool.register_logic_app(logic_app_name, trigger_name)
print(f"Registered logic app '{logic_app_name}' with trigger '{trigger_name}'.")

# Create the specialized "send_email_via_logic_app" function for your agent tools
send_email_func = create_send_email_function(logic_app_tool, logic_app_name)

print(f"✅ Logic App tool configured for: {logic_app_name}")
print(f"🔧 Trigger: {trigger_name}")
print(f"📧 Email function created for workflow integration")

✅ Registered Logic App: agent-logic-apps
✅ Logic App tool configured for: agent-logic-apps
🔧 Trigger: When_a_HTTP_request_is_received
📧 Email function created for workflow integration


In [ ]:
# Prepare the function tools for the agent
functions_to_use: Set = {
    send_email_func,  # This references the AzureLogicAppTool instance via closure
}

# Create the Logic Apps agent with proper toolset
# Create function tool and toolset
functions = FunctionTool(functions=functions_to_use)
toolset = ToolSet()
toolset.add(functions)

# Enable auto function calls - this is key for actual execution
agents_client.enable_auto_function_calls(toolset)

logic_apps_agent = agents_client.create_agent(
    model=model_deployment,
    name="LogicAppsBusinessAgent",
    instructions="""
    You are a business automation assistant that can trigger workflow processes.
    
    Your capabilities:
    - Send email notifications via Logic Apps using send_email_via_logic_app function
    - Trigger alerts for business events
    - Coordinate automated workflows
    
    When sending emails, use the send_email_via_logic_app function with:
    - to: email address
    - subject: email subject line
    - body: email content
    
    Always explain what workflow you're triggering and why.
    """,
    toolset=toolset
)

print(f"🤖 Logic Apps Agent created: {logic_apps_agent.id}")
print(f"🔧 Agent configured with email workflow capabilities")

🤖 Logic Apps Agent created: asst_XTXhL0JYaCRkVPGVQRJYJpVI
🔧 Agent configured with email workflow capabilities


In [ ]:
# Test the Logic Apps agent
thread = agents_client.threads.create()

# Send a test message
message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content="Send a notification email about high sales performance this quarter to samer.elhousseini@microsoft.com"
)

# Create and process an agent run in the thread
run = agents_client.runs.create_and_process(thread_id=thread.id, agent_id=logic_apps_agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

# Get the response
print("📧 Logic Apps Agent Response:")
messages = agents_client.messages.list(thread_id=thread.id)
for msg in messages:
    if msg.text_messages:
        print(f"\n🗨️ {msg.role.upper()}: {msg.text_messages[-1].text.value}")
        print("-" * 80)




📧 Logic Apps Agent Response:

🗨️ Message ID: msg_240PKUG6731W75jBftiLUrYW
 - text message:
I will trigger a workflow to send a notification email regarding the high sales performance for this quarter. This is useful to inform key team members or stakeholders about outstanding results and to recognize the team's efforts.

Could you please specify who should receive this notification email? If not, I can use "team@company.com" as a default recipient. Let me know your preference!
-------------------------------------------------------------------------------- 


🗨️ Message ID: msg_osLMOr7yYpRaRb6r02tmsKjk
 - text message:
Send a notification email about high sales performance this quarter
-------------------------------------------------------------------------------- 



## 🔌 Section 2: Adding Semantic Kernel Plugins

Now we'll add Semantic Kernel plugins for local data processing and create a hybrid agent that combines both capabilities.

In [ ]:
# Create Semantic Kernel plugins for business data processing
class DataProcessingPlugin:
    """Plugin for processing business data locally"""
    
    @kernel_function(
        description="Process sales data and calculate metrics",
        name="process_sales_data"
    )
    def process_sales_data(
        self,
        sales_data: Annotated[str, "JSON string containing sales data"]
    ) -> str:
        """Process sales data and return key metrics"""
        try:
            data = json.loads(sales_data)
            total_revenue = sum(item.get('amount', 0) for item in data)
            avg_deal_size = total_revenue / len(data) if data else 0
            
            metrics = {
                "total_revenue": total_revenue,
                "average_deal_size": avg_deal_size,
                "number_of_deals": len(data),
                "analysis_date": datetime.now().isoformat()
            }
            
            return json.dumps(metrics)
        except Exception as e:
            return f"Error processing sales data: {str(e)}"
    
    @kernel_function(
        description="Generate a business summary report",
        name="generate_summary"
    )
    def generate_summary(
        self,
        metrics: Annotated[str, "JSON string containing business metrics"]
    ) -> str:
        """Generate a formatted business summary"""
        try:
            data = json.loads(metrics)
            
            summary = f"""
📊 BUSINESS PERFORMANCE SUMMARY
================================
💰 Total Revenue: ${data.get('total_revenue', 0):,.2f}
📈 Average Deal Size: ${data.get('average_deal_size', 0):,.2f}
🎯 Number of Deals: {data.get('number_of_deals', 0)}
📅 Analysis Date: {data.get('analysis_date', 'N/A')}

Status: {'🟢 EXCELLENT' if data.get('total_revenue', 0) > 100000 else '🟡 GOOD' if data.get('total_revenue', 0) > 50000 else '🔴 NEEDS ATTENTION'}
"""
            return summary
        except Exception as e:
            return f"Error generating summary: {str(e)}"

# Initialize the plugin
data_plugin = DataProcessingPlugin()
print("✅ Semantic Kernel plugins created successfully!")

In [ ]:
# Convert SK plugins to plain functions for Azure AI Agents
def process_sales_data(sales_data: str) -> str:
    """Process sales data and calculate metrics"""
    try:
        data = json.loads(sales_data)
        total_revenue = sum(item.get('amount', 0) for item in data)
        avg_deal_size = total_revenue / len(data) if data else 0
        
        metrics = {
            "total_revenue": total_revenue,
            "average_deal_size": avg_deal_size,
            "number_of_deals": len(data),
            "analysis_date": datetime.now().isoformat()
        }
        
        return json.dumps(metrics)
    except Exception as e:
        return f"Error processing sales data: {str(e)}"

def generate_summary(metrics: str) -> str:
    """Generate a formatted business summary"""
    try:
        data = json.loads(metrics)
        
        summary = f"""
📊 BUSINESS PERFORMANCE SUMMARY
================================
💰 Total Revenue: ${data.get('total_revenue', 0):,.2f}
📈 Average Deal Size: ${data.get('average_deal_size', 0):,.2f}
🎯 Number of Deals: {data.get('number_of_deals', 0)}
📅 Analysis Date: {data.get('analysis_date', 'N/A')}

Status: {'🟢 EXCELLENT' if data.get('total_revenue', 0) > 100000 else '🟡 GOOD' if data.get('total_revenue', 0) > 50000 else '🔴 NEEDS ATTENTION'}
"""
        return summary
    except Exception as e:
        return f"Error generating summary: {str(e)}"

print("✅ SK plugins converted to functions!")

In [ ]:
# Create hybrid agent with both Logic Apps and SK capabilities
hybrid_functions_to_use: Set = {
    send_email_func,  # Logic Apps function
    process_sales_data,  # SK plugin function 1
    generate_summary  # SK plugin function 2
}

# Create hybrid agent with proper toolset
# Create function tool and toolset
hybrid_functions = FunctionTool(functions=hybrid_functions_to_use)
hybrid_toolset = ToolSet()
hybrid_toolset.add(hybrid_functions)

# Enable auto function calls - this is key for actual execution
agents_client.enable_auto_function_calls(hybrid_toolset)

hybrid_agent = agents_client.create_agent(
    model=model_deployment,
    name="HybridBusinessAgent",
    instructions="""
    You are an intelligent business assistant with hybrid capabilities:
    
    🔄 WORKFLOW AUTOMATION (Logic Apps):
    - Trigger email notifications via send_email_via_logic_app
    - Send business alerts
    - Coordinate automated processes
    
    📊 DATA PROCESSING (Functions):
    - Process sales data locally via process_sales_data
    - Generate business summaries via generate_summary
    - Calculate performance metrics
    
    🎯 HYBRID WORKFLOWS:
    - Analyze data first, then trigger appropriate workflows
    - Combine local processing with cloud automation
    - Provide comprehensive business intelligence
    
    Always explain your actions and coordinate between tools effectively.
    """,
    toolset=hybrid_toolset
)

print(f"🤖 Hybrid Agent created: {hybrid_agent.id}")
print(f"🔧 Tools available: {len(hybrid_toolset)}")

In [ ]:
# Test the hybrid agent with a complex business scenario
thread = agents_client.threads.create()

# Sample sales data for testing
sample_sales_data = json.dumps([
    {"deal_id": "D001", "amount": 25000, "client": "Tech Corp"},
    {"deal_id": "D002", "amount": 45000, "client": "Finance Ltd"},
    {"deal_id": "D003", "amount": 67000, "client": "Retail Inc"},
    {"deal_id": "D004", "amount": 32000, "client": "Healthcare Co"}
])

# Send hybrid request
message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"""
    Please analyze this sales data and then send an appropriate notification:
    
    Sales Data: {sample_sales_data}
    
    1. First process the sales data to calculate metrics
    2. Generate a business summary
    3. If performance is good, trigger a success notification via Logic Apps to samer.elhousseini@microsoft.com
    """
)

print("📤 Sent hybrid request to agent...")
print("🔄 Processing sales data and coordinating workflows...")

In [ ]:
# Execute the hybrid workflow
run = agents_client.runs.create_and_process(thread_id=thread.id, agent_id=hybrid_agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

print("🎉 Hybrid workflow completed!")

In [ ]:
# Display the final results
messages = agents_client.messages.list(thread_id=thread.id)

print("🎯 HYBRID AGENT RESPONSE:")
print("=" * 50)
for msg in messages:
    if msg.text_messages:
        print(f"{msg.role.upper()}: {msg.text_messages[-1].text.value}")
        print("-" * 50)

print("\n✨ WORKFLOW SUMMARY:")
print("📊 Data processed locally using functions")
print("📧 Notifications triggered via Azure Logic Apps")
print("🔗 Hybrid coordination between local AI and cloud automation")

## 🎉 Conclusion

**You've successfully created a hybrid Azure AI Agent that combines:**

### 🔄 Azure Logic Apps Integration
- Used `AzureLogicAppTool` for workflow automation
- Registered Logic Apps with `register_logic_app()` method
- Triggered email notifications and business processes

### 🔌 Semantic Kernel Plugins
- Created local data processing capabilities
- Built custom business intelligence functions
- Processed sales data and generated reports

### 🎯 Hybrid Orchestration
- Combined cloud automation with local AI processing
- Coordinated between multiple tools and workflows
- Created intelligent business automation

**Next Steps:**
- Expand SK plugins for more business logic
- Add more Logic Apps for different workflows
- Implement error handling and monitoring
- Scale to enterprise-level scenarios

---
*Happy automating! 🚀*